In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
data_preprocessed=pd.read_csv('Absenteeism_preprocessed.csv')

In [ ]:
data_preprocessed.head()

,Diseases,Birth_related_issues,Poisoning,Light_issues,Month_Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


CREATE TARGETS

In [ ]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [ ]:
def replacer(number):
  if number>data_preprocessed['Absenteeism Time in Hours'].median():
    a=1
  else:
    a=0
  return a

In [ ]:
data_preprocessed['Excessive Absenteeism']=data_preprocessed['Absenteeism Time in Hours'].apply(replacer)

In [ ]:
data_preprocessed.head()

,Diseases,Birth_related_issues,Poisoning,Light_issues,Month_Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


In [ ]:
#Always import to have a balanced dataset.
#Another way to get the targets table.
#targets= np.where(data_preprocessed['Absenteeism Time in Hours'] >
#data_preprocessed['Absenteeism Time in Hours'].median(),1,0)

In [ ]:
data_with_targets= data_preprocessed.drop(['Absenteeism Time in Hours','Day of the week','Daily Work Load Average','Distance to Work'],axis=1)

In [ ]:
unscaled_inputs=data_with_targets.iloc[:,:-1]

In [ ]:
from sklearn.preprocessing import StandardScaler

absenteeism_scalar = StandardScaler()

In [ ]:
absenteeism_scalar.fit(unscaled_inputs)

StandardScaler()

In [ ]:

scaled_inputs=absenteeism_scalar.transform(unscaled_inputs)

In [ ]:
targets=data_with_targets.iloc[:,-1]

In [ ]:
targets

0      1
1      0
2      0
3      1
4      0
      ..
695    1
696    0
697    1
698    0
699    0
Name: Excessive Absenteeism, Length: 700, dtype: int64

SPLITTING THE TESTING THE DATA

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs,targets, train_size=0.8,random_state=20)

LOGISTIC REGRESSION WITH SKLEARN

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
reg = LogisticRegression()


In [ ]:
reg.fit(x_train,y_train)

LogisticRegression()

In [ ]:
reg.score(x_train,y_train)

0.7839285714285714

In [ ]:
feature_name=unscaled_inputs.columns.values

In [ ]:
#A way to create a dataframe in python
summary_table= pd.DataFrame (columns=['Feature name'], data = feature_name)

In [ ]:
summary_table['Coefficient']=np.transpose(reg.coef_)

In [ ]:
summary_table

,Feature name,Coefficient
0,Diseases,2.069521
1,Birth_related_issues,0.335811
2,Poisoning,1.562039
3,Light_issues,1.312780
4,Month_Value,0.184698
5,Transportation Expense,0.691262
6,Age,-0.198283
7,Body Mass Index,0.324360
8,Education,-0.126656
9,Children,0.370177


In [ ]:
#Adding the intercept to the table.
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept',reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficient
0,Intercept,-0.223756
1,Diseases,2.069521
2,Birth_related_issues,0.335811
3,Poisoning,1.562039
4,Light_issues,1.312780
5,Month_Value,0.184698
6,Transportation Expense,0.691262
7,Age,-0.198283
8,Body Mass Index,0.324360
9,Education,-0.126656


In [ ]:
#Standardized coefficients are basically the coefficients of a regression where all variables have been standardized
#Not a good idea to scale dummy variables.


In [ ]:
summary_table['Odds_Ratio']=np.exp(summary_table.Coefficient)

In [ ]:
summary_table.sort_values('Odds_Ratio',ascending=False)

,Feature name,Coefficient,Odds_Ratio
1,Diseases,2.069521,7.921031
3,Poisoning,1.562039,4.768536
4,Light_issues,1.312780,3.716491
6,Transportation Expense,0.691262,1.996232
10,Children,0.370177,1.447991
2,Birth_related_issues,0.335811,1.399074
8,Body Mass Index,0.324360,1.383144
5,Month_Value,0.184698,1.202855
9,Education,-0.126656,0.881037
7,Age,-0.198283,0.820138


In [ ]:
#Drop Unnecessary Variables


Testing the model

In [ ]:
reg.score(x_test,y_test)

0.7357142857142858

In [ ]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.70804629, 0.29195371],
       [0.57178351, 0.42821649],
       [0.39895299, 0.60104701],
       [0.78714111, 0.21285889],
       [0.06691516, 0.93308484],
       [0.31109284, 0.68890716],
       [0.28582378, 0.71417622],
       [0.08093986, 0.91906014],
       [0.79926793, 0.20073207],
       [0.7496687 , 0.2503313 ],
       [0.46772305, 0.53227695],
       [0.18449682, 0.81550318],
       [0.04086632, 0.95913368],
       [0.75596678, 0.24403322],
       [0.23709315, 0.76290685],
       [0.54140798, 0.45859202],
       [0.53354387, 0.46645613],
       [0.52039975, 0.47960025],
       [0.40507469, 0.59492531],
       [0.02759587, 0.97240413],
       [0.70148836, 0.29851164],
       [0.78714111, 0.21285889],
       [0.40545618, 0.59454382],
       [0.40545618, 0.59454382],
       [0.17391667, 0.82608333],
       [0.75362875, 0.24637125],
       [0.48817233, 0.51182767],
       [0.88053255, 0.11946745],
       [0.13059235, 0.86940765],
       [0.78714111, 0.21285889],
       [0.

In [ ]:
predicted_proba[:,1]

array([0.29195371, 0.42821649, 0.60104701, 0.21285889, 0.93308484,
       0.68890716, 0.71417622, 0.91906014, 0.20073207, 0.2503313 ,
       0.53227695, 0.81550318, 0.95913368, 0.24403322, 0.76290685,
       0.45859202, 0.46645613, 0.47960025, 0.59492531, 0.97240413,
       0.29851164, 0.21285889, 0.59454382, 0.59454382, 0.82608333,
       0.24637125, 0.51182767, 0.11946745, 0.86940765, 0.21285889,
       0.37752575, 0.69562155, 0.71104836, 0.55842034, 0.21285889,
       0.56294675, 0.20932611, 0.81013019, 0.41289739, 0.63001412,
       0.20415726, 0.42151589, 0.22731649, 0.10486653, 0.85723233,
       0.65470153, 0.70629762, 0.29195371, 0.21207006, 0.19572289,
       0.56549874, 0.07774102, 0.68890716, 0.26970997, 0.85980383,
       0.45421224, 0.93050906, 0.21788125, 0.08531744, 0.08952364,
       0.70559435, 0.6774947 , 0.28721976, 0.86078009, 0.18989327,
       0.27063956, 0.01404469, 0.20932611, 0.81292865, 0.28500657,
       0.20932611, 0.06875653, 0.92859838, 0.47793046, 0.63865

Save the model

In [ ]:
import pickle

In [ ]:
with open('model','wb') as file:
  pickle.dump(reg,file)

In [ ]:
with open('scaler','wb') as file:
  pickle.dump(absenteeism_scalar,file)